In [ ]:
#importaciones de modulos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from ipywidgets import interact, widgets, VBox, HBox, Output
from IPython.display import display, clear_output